In [ ]:
#import libraries
import pandas as pd
import torch 

#import other python files
import Configuration

In [ ]:
class InputsAndLabels:
    #Constructor
    def __init__(self, df):
        self.df = df
        self.labels = df[Configuration.DEPENDENT_VAR].values
        self.df = self.df.drop([Configuration.DEPENDENT_VAR],axis=1)
    
    #Creates X vaariable for input to the graph neural Network
    def get_inputs(self):
        inputs = []
        for i in range(len(self.df)):
            inputs.append(list(self.df.iloc[[i],:].values[0]))
        return torch.tensor(inputs)

    #Gets the Y variable, As labels for Training Data
    def get_labels(self):
        temp_labels = [0 for i in range(len(self.labels))]
        for i in range(len(self.labels)):
            if self.labels[i] == 'Yes':
                temp_labels[i] = 1
            else:
                temp_labels[i] = 0
        return torch.tensor(temp_labels)

In [ ]:
class DataPreProcessing:
    def __init__(self):
        self.df = pd.read_csv(Configuration.TRAIN_DATAFILE_PATH)
        self.df = self.df.dropna()
    
    #Process the Hard-Coded Variables
    def process_specific_fields(self):
        self.df = self.df.drop(['CustomerID'],axis=1)
        self.df["HandsetPrice"] = self.df["HandsetPrice"].replace('Unknown', 0)
        self.df['HandsetPrice'] = self.df['HandsetPrice'].astype(int)
        self.df['ServiceArea'] = self.df['ServiceArea'].str.replace('\d+', '')
        self.df['ServiceArea'] = self.df['ServiceArea'].str[:3]
        return self.df

    #Normailize the rows with continuous variables
    def normalize(self):
        self.df[Configuration.CONT_VAR] = self.df[Configuration.CONT_VAR].apply(
            lambda x: (x - x.min()) / (x.max() - x.min()))
        return self.df

    #One Hot encode the catagorical variables
    def one_hot_encode(self):
        self.df = pd.get_dummies(data=self.df, columns=Configuration.CAT_VAR)

    #Save processed Data Files
    def save_processed_file(self):
        self.df.to_csv(Configuration.PROCESSED_FILE_PATH, index=False)
    
    #Applies PCA to the Dataset
    def apply_pca(self):
        pass


In [ ]:
if __name__ == "__main__":
    dataPreProcessing = DataPreProcessing()
    df = dataPreProcessing.process_specific_fields()
    df = dataPreProcessing.one_hot_encode()
    df = dataPreProcessing.normalize()
    dataPreProcessing.save_processed_file()